<a href="https://colab.research.google.com/github/HpDiniz/Artigo-BWAIF-2023-Henrique-Diniz/blob/main/Artigo_BWAIF_2023_Henrique_Diniz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Define a janela de treino em meses: 1 ou 6
janela_treino = 1

# Define qual a categoria será treinada: Geral, Papel, Tijolo ou Hibrido
tipo_interesse = "Hibrido"

# Define se os dados serão coletados ou obtidos de um csv armazenado no github
consume_data_from_csv = True

In [ ]:
import os

# Define as credenciais para utilização do MlFlow via DagsHub
os.environ['MLFLOW_TRACKING_USERNAME'] = ""
os.environ['MLFLOW_TRACKING_PASSWORD'] = ""
os.environ['MLFLOW_TRACKING_PROJECTNAME'] = ""

# 0. Install Dependencies

In [ ]:
import warnings;
warnings.simplefilter('ignore')

from sklearn.exceptions import ConvergenceWarning
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
!pip install pystan --quiet
!pip install statsmodels --quiet
!pip install xgboost==1.6.2 --quiet
!pip install pmdarima --quiet
!pip install mysqlclient --quiet
!pip install psycopg2-binary==2.8.6 --quiet
!pip install mlflow --quiet
!pip install pyngrok --quiet
!pip install unidecode --quiet

In [ ]:
import os
import bs4
import json
import mlflow
import requests
import itertools

import regex as re
import numpy as np
import pandas as pd

from datetime import date
from getpass import getpass
from bs4 import BeautifulSoup
from unidecode import unidecode
from xgboost import XGBRegressor

from sqlalchemy import create_engine
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 1. Read in Data and Process Dates

In [ ]:
this_month = "2023-01"
last_month = "2022-12"

headers = {
    'User-Agent':
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36'
        ' (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'
}

experiment_name = f'{last_month}-{tipo_interesse}-{janela_treino}M'

In [ ]:
def converteData(datas, monthYearOnly):

    new_array = []
    meses = ["Janeiro","Fevereiro","Março","Abril","Maio","Junho","Julho","Agosto","Setembro","Outubro","Novembro","Dezembro"]

    for data in datas:

        item = data.split("/")
        mes = str(meses.index(item[0])+1)
        mes = ("0" + mes)[len(mes)-1:len(mes)+1]

        new_date = item[1] + "-" + mes

        if not monthYearOnly:
            new_date = new_date + "-01 00:00:00"
        
        new_array.append(new_date)
        
    return new_array

def obtem_datas_faltantes(df, date_colun):

    datas_faltantes = []
    start_date = df[date_colun].min()
    end_date = df[date_colun].max()

    while(start_date < end_date):
        date = str(start_date)[0:10]
        df_aux = df[df[date_colun] == date]

        if(len(df_aux) < 1):
            datas_faltantes.append(date)

        start_date = (start_date + relativedelta(days=1))

    return datas_faltantes

def obtem_dados_mercado(indice):

    indice = indice.lower()

    if indice == "igpm":
        indice = "igp-m"

    response = requests.get('https://www.dadosdemercado.com.br/economia/' + indice, headers=headers)
    if response.status_code == 200:
        df_igpm = pd.read_html(response.content, encoding='utf-8')[0]

    anos = list(df_igpm.iloc[:, 0].values)

    timestamp = []
    values = []

    for i in range(len(anos)):
        for m in range(12, 0, -1):
            taxa = str(list(df_igpm.iloc[:, m].values)[i])
            if taxa != '--':
                mes = str(m) if m > 9 else "0" + str(m)
                timestamp.append(str(anos[i]) + "-" + mes)
                values.append(round(float(taxa.replace("%","").replace(",",".")), 2))

    # Create DataFrame
    df_tax = pd.DataFrame({
        'Timestamp': timestamp,
        'Value': values
    })

    df_tax['Value'] = pd.to_numeric(df_tax['Value'], downcast="float")

    return df_tax.replace(0, 0.01) 

def get_all_funds():

    response = requests.get('https://www.fundsexplorer.com.br/ranking', headers=headers)
    if response.status_code == 200:
        df = pd.read_html(response.content, encoding='utf-8')[0]

    idx = df[df['Setor'].isna()].index
    df_funds = df.drop(idx)

    df_funds = df_funds.rename(columns=lambda x: re.sub(r'Código\s*do\s*fundo', 'Ticker', x))
    df_funds = df_funds.rename(columns=lambda x: re.sub(r'Quantidade\s*Ativos', 'QuantidadeAtivos', x))

    col_categorical = ['Ticker','Setor']
    df_funds[col_categorical] = df_funds[col_categorical].astype('category')

    df_funds.sort_values('Ticker', inplace=True)

    df_funds = df_funds.drop_duplicates(subset=['Ticker']).replace('Títulos e Valores Mobiliários','Títulos e Val. Mob.')

    df_funds = df_funds[['Ticker','Setor','QuantidadeAtivos']].reset_index(drop=True)

    return df_funds

def get_close(fund, years):

    df_close = pd.DataFrame()

    start_date = "01-01-" + str(int(pd.to_datetime('today').strftime("%Y")) - years) 
    end_date = pd.to_datetime('today').replace(day=1,hour=0,minute=0,second=0,microsecond=0).strftime("%d-%m-%Y")

    response = requests.get('https://fii-api.infomoney.com.br/api/v1/fii/cotacao/historico/grafico?Ticker='+fund+'&DataInicio='+start_date+'&DataFim='+end_date, headers=headers)

    if not str(response.content) == "b''":

        json_response = json.loads(response.content)

        if 'errors' in json_response:
            print(str(json_response['errors']))
        else:
            df_close = pd.read_json(json.dumps(json_response['dataValor']))

            df_close['Ticker'] = fund
            df_close['Ticker'] = df_close['Ticker'].astype('category')

            df_close.rename(columns={'valor': 'Close'}, inplace = True)

            df_close['Datetime'] = pd.to_datetime(df_close['data'], format='%d-%m-%YT%H:%M:%S')

            df_close.drop(columns={'data'}, inplace = True)
        
    return df_close.replace(0, 0.01) 

def get_dividends(fund, years):

    min_date = str(int(pd.to_datetime('today').strftime("%Y")) - years) + "-01"

    response = requests.get('https://www.fundsexplorer.com.br/funds/' + fund, headers=headers)

    soup = bs4.BeautifulSoup(response.content, "html")
    div = soup.find("div", {"id": "dividends-chart-wrapper"})

    labels = re.findall('"labels":\[.*?\]', str(div))
    dividends = re.findall('"data":\[.*?\]', str(div))

    dividends = json.loads("{" + dividends[0] + "}")['data']
    labels = json.loads("{" + labels[0] + "}")['labels']

    dates = converteData(labels, True)

    result = []
    if len(dates) > 0 and len(dates) == len(dividends):
        for i in range(len(dates)):
            if dates[i] >= min_date:
                result.append({
                    "Ticker": fund,
                    "Datetime": dates[i],
                    "Dividends": round(dividends[i],2)
                })

    df_dividends = pd.DataFrame(result)

    return df_dividends.replace(0, 0.01) 

def get_adress(fundo):

    api_url = "https://fii-api.infomoney.com.br/api/v1/propertie/" + fundo
    response = requests.get(api_url)
    data = []

    if '{' in str(response.content):

        response = response.json()

        for item in response["property"]:

            row = {
                "Ticker": fundo,
                "Tipo": item["type"],
                "Nome": item["name"],
                "DataCompra": item["datePurchase"],
                "ValorAreaBrutaLocavel": item["valueGrossLeasableArea"],
                "Estado": item["state"],
                "Cidade": item["city"],
                "Endereco": item["address"],
                "GoogleMapsLink": item["googleMapsLink"],
                "PercentualPartic": item["percentagePartic"],
                "PecentualVacancia": item["percentVacancy"],
                "PercentualInadimplencia90Dias": item["percent90DayDeliquency"],
                "PercentualFii": item["percentFii"],
                "Latitude": float("NaN"),
                "Longitude": float("NaN")
            }

            cordinates = re.findall("(?<=@)[-]*[\d.]*,-[\d.]*", item['googleMapsLink'])

            if(len(cordinates) > 0):
                cordinates = cordinates[0].split(",")
                row["Latitude"], row["Longitude"] = float(cordinates[0]), float(cordinates[1])
            else:
                
                adress_url = ("https://www.google.com/maps/place/" + item["address"] + "," + item["city"] + "-" + item["state"]).replace(" ", "%20")

                response = requests.get(adress_url)

                cordinates = re.findall("(?<=@)[-]*[\d.]*,-[\d.]*", str(response.content))

                if(len(cordinates) > 0):
                    print("Endereço não encontrado, obtendo Latitude e Longitude aproximada...")
                    cordinates = cordinates[0].split(",")
                    row["Latitude"], row["Longitude"] = float(cordinates[0]), float(cordinates[1])
                else:
                    print("Endereço não encontrado e FALHA ao obter Latitude e Longitude aproximada...")

            data.append(row)

    return pd.DataFrame(data)

def filtra_tipo(df_history, df_funds, tipo):

    tickers = list(df_funds['Ticker'].values)

    if tipo == "Papel":
        tickers = list(df_funds[(df_funds["Setor"] == "Títulos e Val. Mob.")]['Ticker'].values)
    elif tipo == "Tijolo":
        tickers = list(df_funds[(df_funds["Setor"] != "Títulos e Val. Mob.") & (df_funds["Setor"] != "Híbrido")]['Ticker'].values)
    elif tipo == "Hibrido":
        tickers = list(df_funds[(df_funds["Setor"] == "Híbrido")]['Ticker'].values)
    
    return df_history[df_history['Ticker'].isin(tickers)]

def get_month_close(df_close, date):

    year = int(date.split('-')[0])
    month = int(date.split('-')[1])

    start_date = pd.to_datetime('today').replace(year=year, month= month, day=1,hour=0,minute=0,second=0,microsecond=0)
    end_date = (start_date + relativedelta(months=1))

    df_aux = df_close.copy()

    df_aux = df_aux[df_aux['Datetime'] >= start_date]
    df_aux = df_aux[df_aux['Datetime'] < end_date]

    if len(df_aux) > 0:
        return float(df_aux.values[-1][0])
    else:
        return -1

def has_missing_data(df_history):

    min = str(df_history['Datetime'].min())
    max = str(df_history['Datetime'].max())

    year = int(max.split('-')[0])
    month = int(max.split('-')[1])

    start_date = pd.to_datetime('today').replace(year=year, month=month, day=1,hour=0,minute=0,second=0,microsecond=0)

    while str(start_date.strftime("%Y-%m")) != min:

        if not str(start_date.strftime("%Y-%m")) in list(df_history['Datetime']):
            return True

        start_date = (start_date - relativedelta(months=1))

    return False

def get_history(fund, years):

    df_close = get_close(fund, years)
    df_dividends = get_dividends(fund, years)

    df_history = df_dividends.copy()

    if len(df_history) > 0 and len(df_close) > 0:

        new_df = []
        for index, row in df_history.iterrows():

            #print("Procurando 'Close' de: " + row['Datetime'])
            row['Dividends'] = round(row['Dividends'],2)
            row['Close'] = get_month_close(df_close, row['Datetime'])
            new_df.append(row)

        df_history = pd.DataFrame(new_df)

        datas = list(df_history['Datetime'])

        if has_missing_data(df_history):
            print("FII " + fund + " será removido por estar com dados faltantes.")
            df_history = pd.DataFrame()
    
    return df_history

def process_daily_history(df_history, years):

    # Cria um array de índices
    indices = ['Selic','IPCA','IGPM']

    # Obtém o histórico de índices
    df_indices = {}
    for indice in indices:
        df_indices[indice] = obtem_dados_mercado(indice)

    # Obtém o histórico do IFIX
    df_ifix = get_ifix(2)

    # Cria o histórico diário
    df_history_daily = pd.DataFrame()

    for fund in df_history['Ticker'].unique():

        print("Coletando informações de " + fund + "...")

        df_close = get_close(fund, years)

        df_close["Datetime"] = pd.to_datetime(df_close["Datetime"], format="%Y-%m-%d")

        # Preenche os índices mensais
        meses_percorridos = []

        for index, row in df_close.iterrows():
            
            data_mes = str(row['Datetime'])[0:7]
            df_aux = df_history[(df_history['Datetime'] == data_mes) & (df_history['Ticker'] == fund)]

            if len(df_aux) < 1 or data_mes in meses_percorridos:
                continue

            meses_percorridos.append(data_mes)
            df_close.loc[(df_close['Ticker'] == fund) & (df_close["Datetime"].dt.strftime("%Y-%m").eq(data_mes)), "Dividends"] = float(df_aux['Dividends'].values[0])
            
            for indice in indices:
                df_aux = df_indices[indice][df_indices[indice]['Timestamp'] == data_mes]
                df_close.loc[(df_close['Ticker'] == fund) & (df_close["Datetime"].dt.strftime("%Y-%m").eq(data_mes)), indice] = float(df_aux['Value'].values[0])
            
            df_history_daily = df_history_daily.append(df_close[(df_close['Ticker'] == fund) & (df_close["Datetime"].dt.strftime("%Y-%m").eq(data_mes))])

    # Preenche o IFIX em todas as datas do histórico diário
    datas_percorridos = []
    for index, row in df_history_daily.iterrows():
        
        data = str(row["Datetime"])[0:10]

        print("Preenchendo IFIX em " + data + "...")

        if data not in datas_percorridos:

            df_aux = df_ifix[df_ifix['Datetime'] == data]

            if(len(df_aux) > 0):

                df_history_daily.loc[(df_history_daily["Datetime"].dt.strftime("%Y-%m-%d").eq(data)), "IFIX"] = float(df_aux['Close'].values[0])
                datas_percorridos.append(data)

    return df_history_daily

def preenche_historico_faltante(df_history_daily):

    # Percorre todos os ativos do histórico
    for ticker in df_history_daily['Ticker'].unique():

        print("Adicionando dados faltantes de " + ticker + "...")

        # Obtém o histórico específico do ativo
        df_aux = df_history_daily[df_history_daily['Ticker'] == ticker].copy()

        # Obtém a menor e a maior data do histórico do ativo
        start_date = pd.to_datetime(df_aux['Datetime']).min() + relativedelta(days=1)
        end_date = pd.to_datetime(df_aux['Datetime']).max()

        # Percorra todas as datas do intervalo
        while(start_date < end_date):
            
            # Caso não haja algum registro no histórico para a data atual...
            if (len(df_aux[df_aux['Datetime'].dt.strftime("%Y-%m-%d").eq(str(start_date)[0:10])]) < 1):
                
                # Obtém a data de ontém
                ontem = (start_date - relativedelta(days=1))

                # Obtém os registros de ontém
                df_ontem = df_history_daily[(df_history_daily['Ticker'] == ticker) & (df_history_daily['Datetime'].dt.strftime("%Y-%m-%d").eq(str(ontem)[0:10]))]
                
                # Adiciona a data faltante no histórico
                df_history_daily = df_history_daily.append(pd.DataFrame({
                    "Close": df_ontem['Close'].values[0],
                    "Dividends": df_ontem['Dividends'].values[0],
                    "Ticker": [ticker],
                    "Datetime": [start_date],
                    "Selic": df_ontem['Selic'].values[0],
                    "IPCA": df_ontem['IPCA'].values[0],
                    "IGPM": df_ontem['IGPM'].values[0],
                    "IFIX": df_ontem['IFIX'].values[0]
                }))

            # Incrementa a data de início
            start_date = (start_date + relativedelta(days=1))

    # Ordena todos os registros pelo Ticker e Data
    df_history_daily.sort_values(by=['Ticker', 'Datetime'], inplace = True)
    df_history_daily = df_history_daily.reset_index(drop = True)
    return df_history_daily

def process_history(df_funds, years):

    df_adress = pd.DataFrame()
    df_history = pd.DataFrame()
    
    # Percorre a lista de fundos para obter o histórico individual de cada um deles
    for fund in df_funds['Ticker']:

        print("Coletando informações de " + fund + "...")

        df_aux_1 = get_adress(fund)
        df_aux_2 = get_history(fund, years)
        
        df_adress = df_adress.append(df_aux_1)
        df_history = df_history.append(df_aux_2)

        print(str(len(df_aux_2)) + " dados de histórico e " + str(len(df_aux_1)) + " endereços foram encontrados.")

    is_NaN = df_history.isnull()
    row_has_NaN = is_NaN.any(axis=1)
    rows_with_NaN = df_history[row_has_NaN]
    tickers = rows_with_NaN['Ticker'].unique()
    df_history = df_history[~df_history['Ticker'].isin(tickers)]

    df_history = df_history[df_history['Datetime'] <= last_month]
    df_history = df_history.drop_duplicates().replace(np.inf, 0).replace(-np.inf,0).replace(0,0.001)

    for fund in df_history["Ticker"].unique():
        if(len(df_history[df_history["Ticker"] == fund]) < 12):
            df_history = df_history[df_history["Ticker"] != fund]

    a = df_history[df_history['Datetime'] == last_month].Ticker.values
    b = df_history.Ticker.unique()
    intersection = list(set(a) & set(b))
    fundos_faltantes = list(set(a) ^ set(b))

    df_history = df_history[~df_history['Ticker'].isin(fundos_faltantes)]
    
    return df_history, df_adress

def remove_big_variations(df, col_dict):

    drop_indexes = []
    for index, fundo in enumerate(df['Ticker'].unique()):
        df_variacoes = df[df["Ticker"] == fundo]

        for key in col_dict:
            df_variacoes = df_variacoes[(abs(df[key]) >= col_dict[key])]

            if len(df_variacoes) > 0:
                drop_indexes = drop_indexes + list(df[(df["Datetime"] <= df_variacoes["Datetime"].values[-1]) & (df.Ticker == fundo)].index)

    df = df.drop(drop_indexes)
    ticker_before = df["Ticker"].unique()

    # Remove fundos que não possuem pelo menos 20 registros
    for fund in df["Ticker"].unique():
        if(len(df[df["Ticker"] == fund]) < 20):
            df = df[df["Ticker"] != fund]

    print("Tickers removidos: ", set(ticker_before) - set(df["Ticker"].unique()))

    return df

def ajusta_desdobramento(df):
    
    # Desdobramentos obtidos em: https://br.investing.com/stock-split-calendar/
    desdobramentos = {
        "BTCI11": ["2023-01", 9],
        "CYCR11": ["2022-10", 10],
        "EQIR11": ["2022-09", 10],
        "VGIR11": ["2022-09", 10],
        "GALG11": ["2022-08", 10],
        "ARRI11": ["2022-08", 10],
        "VIUR11": ["2022-05", 10],
        "XPSF11": ["2022-05", 10],
        "VIFI11": ["2022-04", 10],
        "GAME11": ["2022-03", 10],
        "BLMR11": ["2021-09", 10],
        "MAXR11": ["2021-04", 19],
        "RMAI11": ["2021-03", 10],
        "FISC11": ["2020-12", 10],
        "PQAG11": ["2020-11", 10]
    }

    for key in desdobramentos:
        if len(df[df["Ticker"] == key]) > 0:
            for index, row in df.iterrows():
                if row["Ticker"] == key and row["Datetime"] < desdobramentos[key][0]:
                    df.at[index,'Close'] = round(row['Close']/ desdobramentos[key][1],2)

def getSectorMeans(df_funds, df_history):

    df_setores = pd.DataFrame(({
        'Setor':[],
        'Datetime':[],
        'DividendsChangeMean' :[],
        'CloseChangeMean':[],
        'DividendYieldChangeMean':[],
        'DividendsChangeMean6M' :[],
        'CloseChangeMean6M':[],
        'DividendYieldChangeMean6M':[]
    }))

    for setor in df_funds["Setor"].unique():

        setor_tickers = df_funds[df_funds["Setor"] == setor]["Ticker"].values

        df_sector = df_history[df_history["Ticker"].isin(setor_tickers)]
        min_date = pd.to_datetime(df_sector["Datetime"].min()).replace(day=1)
        max_date = pd.to_datetime(df_sector["Datetime"].max()).replace(day=1)

        while min_date <= max_date:

            date = (min_date).strftime("%Y-%m")

            df_setores = df_setores.append({
                'Setor': setor, 
                'Datetime':date, 
                'DividendsChangeMean': df_sector[df_sector["Datetime"] == date]["DividendsChange"].mean(), 
                'CloseChangeMean': df_sector[df_sector["Datetime"] == date]["CloseChange"].mean(), 
                'DividendYieldChangeMean': df_sector[df_sector["Datetime"] == date]["DividendYieldChange"].mean(),
                'DividendsChangeMean6M': df_sector[df_sector["Datetime"] == date]["DividendsChange6M"].mean(),
                'CloseChangeMean6M': df_sector[df_sector["Datetime"] == date]["CloseChange6M"].mean(),
                'DividendYieldChangeMean6M': df_sector[df_sector["Datetime"] == date]["DividendYieldChange6M"].mean(),
            }, ignore_index=True)

            min_date = min_date + relativedelta(months=1)
    
    return df_setores

def get_ifix(years):

    df_ifix = pd.DataFrame()
    final_date = pd.to_datetime('today').strftime("%d-%m-%Y").replace("-","%2F")
    initial_date = str(int(pd.to_datetime('today').strftime("%Y")) - years) + "-01-01"

    headers_aux = {
        'authority':'www.infomoney.com.br',
        'accept':'application/json, text/javascript, */*; q=0.01',
        'accept-language':'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
        'content-type':'application/x-www-form-urlencoded; charset=UTF-8',
        'authority': 'www.infomoney.com.br',
        'origin':'https://www.infomoney.com.br',
        'referer':'https://www.infomoney.com.br/cotacoes/b3/indice/ifix/historico/',
    }

    body_aux = 'page=0&numberItems=99999&initialDate='+initial_date+'&finalDate='+final_date+'&symbol=IFIX'

    response = requests.post('https://www.infomoney.com.br/wp-json/infomoney/v1/quotes/history', headers=headers_aux,  data=body_aux)

    if not str(response.content) == "b''":

        json_response = json.loads(response.content)

        jobject = []
        for obj in json_response:
            jobject.append({
                'data': obj[0]['display'],
                'Close': obj[2]
            })

        df_ifix = pd.DataFrame(jobject)
        df_ifix['Datetime'] = pd.to_datetime(df_ifix['data'], format='%d/%m/%Y')
        df_ifix.drop(columns={'data'}, inplace = True)

    return df_ifix

def add_pct_changes(df_history):

    # Cria o DataFrame a ser aprimorado
    df_improved = df_history.copy()

    # Remove fundos que não possuem dados do mês anterior
    df_improved = df_improved[df_improved['Datetime'] <= last_month]
    df_improved = df_improved.drop_duplicates().replace(np.inf, 0).replace(-np.inf,0).replace(0,0.001)

    # Remove fundos que não possuem pelo menos 12 registros
    for fund in df_improved["Ticker"].unique():
        if(len(df_improved[df_improved["Ticker"] == fund]) < 12):
            df_improved = df_improved[df_improved["Ticker"] != fund]

    # Normaliza os dados que sofreram desdobramento
    ajusta_desdobramento(df_improved)
    df_improved = df_improved.replace(np.inf, 0).replace(-np.inf,0).replace(0,0.001)

    # Cria a coluna DividendYield
    df_improved['DividendYield'] = round(100*df_improved['Dividends']/df_improved['Close'],6)

    # Cria novas colunas contendo a variação de valores ao longo dos meses
    for index, fundo in enumerate(df_improved['Ticker'].unique()):
        df_improved.loc[df_improved.Ticker == fundo, 'DividendsChange'] = round(df_improved[df_improved.Ticker == fundo]['Dividends'].pct_change(),6)
        df_improved.loc[df_improved.Ticker == fundo, 'CloseChange'] = round(df_improved[df_improved.Ticker == fundo]['Close'].pct_change(),6)
        df_improved.loc[df_improved.Ticker == fundo, 'DividendYieldChange'] = round(df_improved[df_improved.Ticker == fundo]['DividendYield'].pct_change(),6)
        df_improved.loc[df_improved.Ticker == fundo, 'DividendsChange6M'] = round(df_improved[df_improved.Ticker == fundo]['Dividends'].pct_change(periods=6),6)
        df_improved.loc[df_improved.Ticker == fundo, 'CloseChange6M'] = round(df_improved[df_improved.Ticker == fundo]['Close'].pct_change(periods=6),6)
        df_improved.loc[df_improved.Ticker == fundo, 'DividendYieldChange6M'] = round(df_improved[df_improved.Ticker == fundo]['DividendYield'].pct_change(periods=6),6)

    # Procura no DataFrame registros com variações muito discrepantes
    df_improved = remove_big_variations(df_improved, {'DividendsChange': 500, 'CloseChange': 0.35})

    return df_improved

def improve_history(df_history, df_funds):

    df_improved = add_pct_changes(df_history)

    df_sectors = getSectorMeans(df_funds, df_improved)

    # Cria um array de índices
    indices = ['Selic','IPCA','IGPM']

    # Obtém o histórico do IFIX
    df_ifix = get_ifix(2)
    
    # Obtém o histórico de índices
    df_indices = {}
    for indice in indices:
        df_indices[indice] = obtem_dados_mercado(indice)

    # Cria as colunas dos índices
    for indice in indices:
        df_improved[indice] = float("NaN")
                    
    # Insere preço dos índices e a média do setor ao longo do tempo
    for index, fundo in enumerate(df_improved['Ticker'].unique()):

        print(str(index+1) + "/" + str(len(df_improved['Ticker'].unique())))

        sector = df_funds[df_funds["Ticker"] == fundo]["Setor"].values[0]

        for data in df_improved['Datetime']:

            df_improved.loc[(df_improved.Ticker == fundo) & (df_improved.Datetime == data), "IFIX"] = get_month_close(df_ifix, data)

            sector_values = df_sectors[(df_sectors["Datetime"] == data) & (df_sectors["Setor"] == sector)]

            if len(sector_values) > 0:
                for mean_col in ["DividendsChangeMean", "CloseChangeMean", "DividendYieldChangeMean"]:
                    mean_value = sector_values[mean_col].values[0]
                    df_improved.loc[(df_improved.Ticker == fundo) & (df_improved.Datetime == data), "Sector" + mean_col] = float(mean_value)

            for indice in indices:
                indice_values = df_indices[indice][df_indices[indice].Timestamp == data]['Value'].values
                if len(indice_values) > 0:
                    df_improved.loc[(df_improved.Ticker == fundo) & (df_improved.Datetime == data), indice] = float(indice_values[0])


    df_improved['IFIX'] = df_improved['IFIX'].astype(float)
    df_improved = remove_big_variations(df_improved, {'CloseChange': 0.35})

    return df_improved

In [ ]:
if consume_data_from_csv:

    # Os dados serão consumidos de dados previamente coletados e armazenados em GitHub
    
    df_funds = pd.read_csv("https://raw.githubusercontent.com/HpDiniz/Artigo-BWAIF-2023-Henrique-Diniz/main/df_funds_2022-12.csv", encoding ='iso-8859-1', sep=";").set_index('Unnamed: 0')
    df_funds.index.name = None

    df_history = pd.read_csv("https://raw.githubusercontent.com/HpDiniz/Artigo-BWAIF-2023-Henrique-Diniz/main/df_history_2022-12.csv", encoding ='iso-8859-1', sep=";").set_index('Unnamed: 0')
    df_history.index.name = None

    df_adress = pd.read_csv("https://raw.githubusercontent.com/HpDiniz/Artigo-BWAIF-2023-Henrique-Diniz/main/df_adress_2022-12.csv", sep=",").set_index('Unnamed: 0')
    df_adress.index.name = None

else:

    # Os dados serão coletados através de web scraping em diversas fontes distintas

    df_funds = get_all_funds()

    df_history, df_adress = process_history(df_funds, 2)

    df_history = improve_history(df_history, df_funds)

In [ ]:
# Remove fundos imobiliários que possuírem 3 meses seguidos sem variação no preço da cota

invalid_tickers = list(df_history[(df_history['CloseChange'] == 0.0) & (df_history['CloseChange'].shift(1) == 0.0) & (df_history['CloseChange'].shift(2) == 0.0)].Ticker.unique())
df_history = df_history[~(df_history['Ticker'].isin(invalid_tickers))]

In [ ]:
df_funds.head()

In [ ]:
df_history.head()

In [ ]:
df_adress.head()

In [ ]:
# Exibe informações sobre os dados obtidos

print(f"{len(df_history.Ticker.unique())} FIIs restaram na análise:")

for tipo in ["Papel", "Hibrido", "Tijolo"]:
    print(f"- {len(filtra_tipo(df_history, df_funds, tipo).Ticker.unique())} FIIs de {tipo}.")

print(f"\n{len(df_adress)} endereços de FIIs foram encontrados.")

percent = df_adress['Latitude'].isnull().sum()/(len(df_adress))*100
print("%.2f%% dos endereços estão sem Latitude e Longitude." % percent)

# 2. Data Prediction

In [ ]:
def calculate_mae_rmse(abs_errors):

    # Garante que teremos apenas 6 casas decimais
    abs_errors = [round(x,6) for x in abs_errors]

    # Calcular o Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_true=np.zeros_like(abs_errors), y_pred=np.array(abs_errors))

    # Calcular o Root Mean Squared Error (RMSE)
    rmse = mean_squared_error(y_true=np.zeros_like(abs_errors), y_pred=np.array(abs_errors), squared=False)

    return round(mae, 6), round(rmse, 6)

def upload_errors(pred_col, train_cols, strategy, sectors, abs_errors, params = None):

    mae, rmse = calculate_mae_rmse(abs_errors)

    with mlflow.start_run(run_name=train_cols):   
        
        df_abs = pd.DataFrame(abs_errors, columns = ['errors'])

        # Parameters
        mlflow.log_param("pred_col", unidecode(pred_col))
        mlflow.log_param("train_cols", unidecode(train_cols))
        mlflow.log_param("strategy", unidecode(strategy))
        mlflow.log_param("sector", unidecode(sectors))

        # Error Metrics
        mlflow.log_metric("MAE", round(mae, 6))
        mlflow.log_metric("RMSE", round(rmse, 6))
        mlflow.log_metric("mean", df_abs.describe().values[1][0])
        mlflow.log_metric("std", df_abs.describe().values[2][0])
        mlflow.log_metric("min", df_abs.describe().values[3][0])
        mlflow.log_metric("25 pct.", df_abs.describe().values[4][0])
        mlflow.log_metric("50 pct.", df_abs.describe().values[5][0])
        mlflow.log_metric("75 pct.", df_abs.describe().values[6][0])
        mlflow.log_metric("max", df_abs.describe().values[7][0])

        # Machine Learning Params
        mlflow.log_metric("n_estimators", 0.0 if params == None else params["n_estimators"]) 
        mlflow.log_metric("learning_rate", 0.0 if params == None else params["learning_rate"]) 
        mlflow.log_metric("max_depth", 0.0 if params == None else params["max_depth"]) 
        mlflow.log_metric("min_child_weight", 0.0 if params == None else params["min_child_weight"]) 
        mlflow.log_metric("colsample_bytree", 0.0 if params == None else params["colsample_bytree"])

def get_experiments_result(experiment_name, sort_column = ""):

    df = mlflow.search_runs([mlflow_experiment._experiment_id])

    if(len(df) > 0):
        if sort_column in df.columns:
            return df.sort_values(by=sort_column, ascending=True)
        else:
            return df

    return df

def is_mlfow_configured():
    return not (os.environ['MLFLOW_TRACKING_USERNAME'] == "" or os.environ['MLFLOW_TRACKING_PASSWORD'] == "" or os.environ['MLFLOW_TRACKING_PROJECTNAME'] == "")

def get_possibilities(target_column, training_columns):

    possibilities = []
    for L in range(len(training_columns) + 1):
        for subset in itertools.combinations(training_columns, L):
            possibilities.append([target_column] + list(subset))

    possibilities.reverse()

    return list(filter(lambda x: len(x) > 4, possibilities)) 

def train_test_split(data, perc):

    data = data.values
    n = int(len(data) * (1 - perc))
    return data[:n], data[n:]

def arima_predict(df_history, fundo, pred_col, pred_index = 1):

    df = df_history[df_history['Ticker'] == fundo].copy()
    df['Target'] = df[pred_col].shift(-pred_index)
    df = df[[pred_col, 'Target']]
    df.dropna(inplace=True)

    y_pred = []
    train, test = train_test_split(df, 0.4) # 60% de treino

    # cria a variável history
    history = [x[0] for x in train]

    for i in range(len(test)):
        test_X, test_y = test[i, :-1], test[i, -1]

        # O arima deverá considerar apenas o valor da coluna principal
        model = SARIMAX(history, order=(1,1,1), maxiter=1000)
        resultado_sarimax = model.fit()

        # Obtém a predição de {pred_index} meses à frente
        output = resultado_sarimax.get_forecast(steps=pred_index)

        # Obtém a predição do mês de interesse
        pred = output.predicted_mean[pred_index-1]

        y_pred.append(pred)
        history.append(test[i][0])

    y_real = test[:, -1]

    yhat = round(y_pred[0],2)
    abs_error = list(np.abs(y_real - y_pred))

    return yhat, abs_error

def model_predict(train, test_X, val_X, val_y, model):

    train = np.array(train)
    val_y = np.array([val_y])
    val_X = np.array([val_X])
    test_X = np.array([test_X])

    X, y = train[:, :-1], train[:, -1]

    if(len(val_X) > 0):
        model.fit(X, y, eval_set=[(X,y),(val_X,val_y)], verbose=0)
    else:
        model.fit(X, y)
    
    pred = model.predict(test_X)

    return pred[0]

def machinelearn_predict(df_history, fundo, pred_col, train_cols, params, pred_index = 1):
 
    df = df_history[df_history['Ticker'] == fundo][train_cols].copy()
    df["Target"] = df[pred_col].shift(-pred_index)
    df.dropna(inplace=True)

    model = None
    y_pred = []
    train, test = train_test_split(df, 0.4) # 60% de treino

    history = [x for x in train]

    for i in range(len(test) - 1):

        val_X, val_y = test[i, :-1], test[i, -1]
        test_X, test_y = test[i + 1, :-1], test[i + 1, -1]

        model = XGBRegressor()
        model.set_params(**params)

        pred = model_predict(history, test_X, val_X, val_y, model)

        y_pred.append(pred)

        history.append(test[i])
    
    y_real = test[1:, -1]

    yhat = round(y_pred[0],2)
    abs_error = list(np.abs(y_real - y_pred))

    return yhat, abs_error

In [ ]:
existent_experiments = []

# Trecho para obter os experimentos que já estão no mlflow
if is_mlfow_configured():

    mlflow.set_tracking_uri(f'https://dagshub.com/' + os.environ['MLFLOW_TRACKING_USERNAME'] + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + '.mlflow')
    mlflow_experiment = mlflow.set_experiment(experiment_name)

    try:
        existent_experiments = list(get_experiments_result(experiment_name)["params.train_cols"])
    except:
        print("Nenhum experimento foi encontrado")

In [ ]:
# Trecho para obter as combinações de colunas de acordo com a janela de treino
pred_column = 'CloseChange'
possibilities = get_possibilities(pred_column, (['Close','DividendYield','DividendsChange','DividendYieldChange','SectorDividendsChangeMean','SectorCloseChangeMean','Selic','IPCA','IGPM','IFIX']))

if janela_treino == 6:
    pred_column = 'CloseChange6M'
    possibilities = get_possibilities(pred_column, (['Close','DividendYield','DividendsChange6M','DividendYieldChange6M','SectorDividendsChangeMean','SectorCloseChangeMean','Selic','IPCA','IGPM','IFIX']))

print(str(len(possibilities)) + " possibilidades serão avaliadas")

In [ ]:
# Obtem os Tickers de cada tipo
papel = list(filtra_tipo(df_history, df_funds, 'Papel').Ticker.unique())
tijolo = list(filtra_tipo(df_history, df_funds, 'Tijolo').Ticker.unique())
hibrido = list(filtra_tipo(df_history, df_funds, 'Hibrido').Ticker.unique())

In [ ]:
# Filtra o dataframe de acordo com o tipo que se deseja treinar
df_history = filtra_tipo(df_history, df_funds, tipo_interesse)
sectors = pd.merge(df_funds, df_history, on='Ticker')["Setor"].unique()

In [ ]:
params = {"n_estimators" : 4000, "early_stopping_rounds" : 100, "learning_rate": 0.1, "max_depth": 6,"min_child_weight" : 1,"colsample_bytree" : 0.4}

# Realiza o treino de todas as combinações com XGboost
for j, array_possibility in enumerate(possibilities):
    
    abs_errors = []
    string_possibility =  ", ".join(array_possibility)

    print('Calculating "'+ string_possibility + '": ' + str(j+1) + '/' + str(len(possibilities)))

    if string_possibility not in existent_experiments:

        for i, fundo in enumerate(df_history["Ticker"].unique()):

            print("Calculating errors of "+ fundo + ": " + str(i+1) + "/" + str(len(df_history["Ticker"].unique())))

            prediction, abs_err = machinelearn_predict(df_history, fundo, pred_column, array_possibility, params, janela_treino)
            abs_errors = abs_errors + abs_err

        print(f'(MAE, RMSE): {calculate_mae_rmse(abs_errors)}\n')

        if is_mlfow_configured():
            upload_errors(pred_column, string_possibility, 'Xgboost', ', '.join(sectors), abs_errors, params)

In [ ]:
abs_errors = []

# Realiza o treino do ARIMA
if pred_column not in existent_experiments:

    print('Calculating ARIMA...')

    for i, fundo in enumerate(df_history["Ticker"].unique()):

        print("Calculating errors of "+ fundo + ": " + str(i+1) + "/" + str(len(df_history["Ticker"].unique())))

        prediction, abs_err = arima_predict(df_history, fundo, pred_column, janela_treino)
        abs_errors = abs_errors + abs_err

    print(f'(MAE, RMSE): {calculate_mae_rmse(abs_errors)}\n')
    
    if is_mlfow_configured():
        upload_errors(pred_column, pred_column, 'Arima', ', '.join(sectors), abs_errors)